In [1]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.svm import LinearSVC
from nltk.classify import SklearnClassifier
from sklearn.neighbors import KNeighborsClassifier
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten,Conv1D,MaxPool1D

from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
import sklearn
from sklearn.metrics import classification_report
from joblib import dump, load
import queue
import pandas as pd
import numpy as np
import glob, os, sys
pd.options.mode.chained_assignment = None

e:\anaconda\envs\Adversarial_attack\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
with open('./Dataset/csv/Attack_merge.csv', newline='') as csvfile:
        rows = pd.read_csv(csvfile,header=None)
        y = rows[10]
        x = rows.drop([10], axis=1)
        X_train, X_test, y_train, y_test = train_test_split(x, y , test_size=0.2)
    
if not os.path.exists("./models"):
    os.mkdir("./models")

In [3]:
model = DecisionTreeClassifier()
model.fit(X_train, y_train)
print(model.score(X_test, y_test))
dump(model, './models/DT.joblib')
prediction = model.predict(X_test)
prediction_prob1 = model.predict_proba(X_test)
print(prediction_prob1[0:,1])
print(classification_report(y_test,prediction))

0.9710932260228035
[1. 1. 0. ... 1. 1. 1.]
              precision    recall  f1-score   support

           0       0.97      0.98      0.97     22917
           1       0.97      0.97      0.97     21813

    accuracy                           0.97     44730
   macro avg       0.97      0.97      0.97     44730
weighted avg       0.97      0.97      0.97     44730



In [4]:
model = RandomForestClassifier(n_estimators=20, max_depth=4)
model.fit(X_train, y_train)
print(model.score(X_test, y_test))
dump(model, './models/RF.joblib')
prediction = model.predict(X_test)
prediction_prob2 = model.predict_proba(X_test)
print(prediction_prob2[0:,1])
print(classification_report(y_test,prediction))

0.9485580147551979
[0.91141263 0.99584606 0.26046839 ... 0.88290394 0.88290394 0.91141263]
              precision    recall  f1-score   support

           0       0.92      0.99      0.95     22917
           1       0.99      0.91      0.95     21813

    accuracy                           0.95     44730
   macro avg       0.95      0.95      0.95     44730
weighted avg       0.95      0.95      0.95     44730



In [9]:
a = np.array([prediction_prob1[0:,1],prediction_prob2[0:,1]])
print(a)
a = np.average(a,axis=0)
print(a)
a = np.rint(a)
print(a)

[[1.         1.         0.         ... 1.         1.         1.        ]
 [0.91141263 0.99584606 0.26046839 ... 0.88290394 0.88290394 0.91141263]]
[0.95570631 0.99792303 0.1302342  ... 0.94145197 0.94145197 0.95570631]
[1. 1. 0. ... 1. 1. 1.]


In [4]:
model = LogisticRegression(C=3)
model.fit(X_train, y_train)
print(model.score(X_test, y_test))
dump(model, './models/LR.joblib')
prediction = model.predict(X_test)
print(classification_report(y_test,prediction))

0.9075788061703555
              precision    recall  f1-score   support

           0       0.91      0.91      0.91     22799
           1       0.90      0.91      0.91     21931

    accuracy                           0.91     44730
   macro avg       0.91      0.91      0.91     44730
weighted avg       0.91      0.91      0.91     44730



In [16]:
model = XGBClassifier(booster="gbtree", min_split_loss=7.100747859845302e-07, alpha=0.0010277375135306342, 
                        max_depth=9, eta=0.6737504946980999, gamma=6.101266632438708e-07, grow_policy="lossguide")
model.fit(X_train, y_train)
print(model.score(X_test, y_test))
dump(model, './models/XGB.joblib')
prediction = model.predict(X_test)
print(classification_report(y_test,prediction))

e:\anaconda\envs\Adversarial_attack\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
e:\anaconda\envs\Adversarial_attack\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[01:54:40] WARNING: D:\bld\xgboost-split_1645118015404\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0.9771294433266264
[9.9997020e-01 2.0607738e-02 9.9999404e-01 ... 5.3720969e-06 9.9995923e-01
 4.8870438e-06]
              precision    recall  f1-score   support

           0       0.98      0.97      0.98     22922
           1       0.97      0.98      0.98     21808

    accuracy                           0.98     44730
   macro avg       0.98      0.98      0.98     44730
weighted avg       0.98      0.98      0.98     44730



e:\anaconda\envs\Adversarial_attack\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


In [5]:
model = LinearSVC(C = 2, verbose=False)
model.fit(X_train, y_train)
print(model.score(X_test, y_test))
dump(model, './models/SVM.joblib')
prediction = model.predict(X_test)
prediction_prob = model._predict_proba_lr(X_test)
print(prediction_prob[0:,1])
print(classification_report(y_test,prediction))

0.898859825620389
[0.15382529 0.30297063 0.47198146 ... 0.14430225 0.47160139 0.63542526]
              precision    recall  f1-score   support

           0       0.91      0.89      0.90     22831
           1       0.89      0.91      0.90     21899

    accuracy                           0.90     44730
   macro avg       0.90      0.90      0.90     44730
weighted avg       0.90      0.90      0.90     44730



e:\anaconda\envs\Adversarial_attack\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [6]:
model = KNeighborsClassifier()
model.fit(X_train, y_train)
print(model.score(X_test, y_test))
dump(model, './models/KNN.joblib')
prediction = model.predict(X_test)
print(prediction)
print(classification_report(y_test,prediction))

0.9753856472166331
[0 0 0 ... 1 1 0]
              precision    recall  f1-score   support

           0       0.98      0.97      0.98     22652
           1       0.97      0.98      0.98     22078

    accuracy                           0.98     44730
   macro avg       0.98      0.98      0.98     44730
weighted avg       0.98      0.98      0.98     44730



In [14]:
batch_size = 32
X_train = np.array(X_train)
y_train = np.array(y_train)

earlyStopping = EarlyStopping(monitor='val_loss', patience=3, verbose=0, mode='min')
mcp_save = ModelCheckpoint('./models/DNN4_Best_Model.h5', save_weights_only=False, save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, verbose=1, epsilon=1e-4, mode='min')

dnn4 = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer(input_shape=(X_train.shape[1],)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation = 'relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.01),
    tf.keras.layers.Dense(8, activation = 'relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.01),
    tf.keras.layers.Dense(4, activation = 'relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.01),
    tf.keras.layers.Dense(2, activation='softmax')
])

dnn4.compile(optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']);

dnn4.fit(X_train, y_train, batch_size=batch_size, epochs=10, verbose=1, callbacks=[earlyStopping, mcp_save, reduce_lr_loss], validation_split=0.25)
model = tf.keras.models.load_model('./models/DNN4.h5')

Epoch 1/10


KeyboardInterrupt: 

In [3]:
#model.save("./models/DNN4.h5")
model = tf.keras.models.load_model('./models/DNN4.h5')

In [8]:
preds = model.predict(X_test)
pred_labels = np.rint(preds)
accuracy = sklearn.metrics.accuracy_score(y_test, pred_labels[0:,1])
f1 = sklearn.metrics.f1_score(y_test, pred_labels[0:,1], average="binary")
prec = sklearn.metrics.precision_score(y_test, pred_labels[0:,1], average="binary")
recall = sklearn.metrics.recall_score(y_test, pred_labels[0:,1], average="binary")
print("accuracy: " + str(accuracy))
print("f1: " + str(f1))
print("prec: " + str(prec))
print("recall: " + str(recall))

e:\anaconda\envs\Adversarial_attack\lib\site-packages\tensorflow\python\keras\engine\sequential.py:430: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


[[5.7736558e-01 4.2263442e-01]
 [1.4126042e-04 9.9985874e-01]
 [1.3915132e-02 9.8608482e-01]
 ...
 [9.9870741e-01 1.2926341e-03]
 [9.9930048e-01 6.9949619e-04]
 [9.9930012e-01 6.9987692e-04]]
[4.2263442e-01 9.9985874e-01 9.8608482e-01 ... 1.2926341e-03 6.9949619e-04
 6.9987692e-04]
accuracy: 0.9213056114464565
f1: 0.9185901290531479
prec: 0.9301203690693645
recall: 0.9073422579613469


In [9]:
y_train_ex = tf.expand_dims(y_train, axis=1)
X_train_ex = tf.expand_dims(X_train, axis=2)
y_test_ex = tf.expand_dims(y_test, axis=1)
X_test_ex = tf.expand_dims(X_test, axis=2)
y_train_np = np.array(y_train_ex)
X_train_np = np.array(X_train_ex)
y_test_np = np.array(y_test_ex)
X_test_np = np.array(X_test_ex)

In [10]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Embedding, GlobalAveragePooling1D
epochs = 100
batch_size = 256

model = Sequential()
model.add(Conv1D(filters=100, kernel_size=5, padding='same',activation='relu',strides=1, input_shape=(X_train_ex.shape[1],1))) 
model.add(Dropout(0.2))
model.add(Conv1D(filters=200, kernel_size=5, padding='same', activation='relu',strides=1))
model.add(Dropout(0.2))
model.add(Conv1D(filters=400, kernel_size=10, padding='same', activation='relu',strides=1))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer= 'adam', metrics=['accuracy'])
model.summary()

model.fit(X_train_ex, y_train_np, epochs=epochs, batch_size=batch_size,) 

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_6 (Conv1D)            (None, 10, 100)           600       
_________________________________________________________________
dropout_10 (Dropout)         (None, 10, 100)           0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 10, 200)           100200    
_________________________________________________________________
dropout_11 (Dropout)         (None, 10, 200)           0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 10, 400)           800400    
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 5, 400)            0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 2000)             

In [10]:
#model.save("./models/CNN.h5")
model = tf.keras.models.load_model("./models/CNN.h5")

In [13]:
preds = model.predict(X_test_ex)
pred_labels = np.rint(preds)
accuracy = sklearn.metrics.accuracy_score(y_test_ex, pred_labels)
f1 = sklearn.metrics.f1_score(y_test_ex, pred_labels, average="binary")
prec = sklearn.metrics.precision_score(y_test_ex, pred_labels, average="binary")
recall = sklearn.metrics.recall_score(y_test_ex, pred_labels, average="binary")
print("accuracy: " + str(accuracy))
print("f1: " + str(f1))
print("prec: " + str(prec))
print("recall: " + str(recall))

accuracy: 0.9300022356360385
f1: 0.9269294499288199
prec: 0.9473809750977961
recall: 0.9073422579613469


In [5]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

In [6]:
regressor = Sequential()
regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train_np.shape[1], X_train_np.shape[2])))
regressor.add(Dropout(0.2))

# Adding a second LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a third LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a fourth LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))
regressor.add(Dense(1, activation='sigmoid'))
regressor.compile(loss='binary_crossentropy', optimizer= 'adam', metrics=['accuracy'])
regressor.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 10, 50)            10400     
_________________________________________________________________
dropout (Dropout)            (None, 10, 50)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 10, 50)            20200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 50)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 10, 50)            20200     
_________________________________________________________________
dropout_2 (Dropout)          (None, 10, 50)            0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 50)               

In [10]:

# 進行訓練
regressor.fit(X_train_ex, y_train_ex, epochs = 20, batch_size = 32)

Epoch 1/20
5592/5592 [==============================] - 69s 11ms/step - loss: 0.3544 - accuracy: 0.9227
Epoch 2/20
5592/5592 [==============================] - 63s 11ms/step - loss: 0.1407 - accuracy: 0.9546
Epoch 3/20
5592/5592 [==============================] - 64s 11ms/step - loss: 0.1009 - accuracy: 0.9626
Epoch 4/20
5592/5592 [==============================] - 64s 11ms/step - loss: 0.0910 - accuracy: 0.9641
Epoch 5/20
5592/5592 [==============================] - 66s 12ms/step - loss: 0.0876 - accuracy: 0.9649
Epoch 6/20
5592/5592 [==============================] - 64s 11ms/step - loss: 0.0891 - accuracy: 0.9630
Epoch 7/20
5592/5592 [==============================] - 64s 11ms/step - loss: 0.0860 - accuracy: 0.9647
Epoch 8/20
5592/5592 [==============================] - 63s 11ms/step - loss: 0.0828 - accuracy: 0.9654
Epoch 9/20
5592/5592 [==============================] - 64s 11ms/step - loss: 0.0835 - accuracy: 0.9648
Epoch 10/20
5592/5592 [==============================] - 63s 11m

In [16]:
#regressor.save("./models/LSTM.h5")
model = tf.keras.models.load_model("./models/LSTM.h5")

In [17]:
preds = model.predict(X_test_ex)
print(model.predict_proba(X_test_ex))
pred_labels = np.rint(preds)
accuracy = sklearn.metrics.accuracy_score(y_test_ex, pred_labels)
f1 = sklearn.metrics.f1_score(y_test_ex, pred_labels, average="binary")
prec = sklearn.metrics.precision_score(y_test_ex, pred_labels, average="binary")
recall = sklearn.metrics.recall_score(y_test_ex, pred_labels, average="binary")
print("accuracy: " + str(accuracy))
print("f1: " + str(f1))
print("prec: " + str(prec))
print("recall: " + str(recall))

e:\anaconda\envs\Adversarial_attack\lib\site-packages\tensorflow\python\keras\engine\sequential.py:430: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


[[7.5782835e-01]
 [9.9833703e-01]
 [9.9833858e-01]
 ...
 [8.8761508e-04]
 [8.9969923e-04]
 [8.9435477e-04]]
accuracy: 0.9684328191370445
f1: 0.9676606660253767
prec: 0.9701492537313433
recall: 0.9651848129026362
